In [2]:
# 1. Advanced CRUD Operations
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

# Koneksi ke MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

# Bersihkan koleksi agar tidak ada data lama
courses_collection.delete_many({})

# === BULK OPERATION ===
operations = [
    # INSERT: Menambahkan beberapa course baru
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}),

    # UPDATE: Misalnya ubah jumlah enrollments untuk beberapa course
    UpdateOne({'course': 'Math 101'}, {'$set': {'enrollments': 35}}),
    UpdateOne({'course': 'CS 102'}, {'$inc': {'enrollments': 5}}),  # tambah 5 siswa

    # DELETE: Hapus satu course yang tidak lagi aktif
    DeleteOne({'course': 'History 201'})
]

# Jalankan operasi secara bulk
result = courses_collection.bulk_write(operations)
print("Eksekusi Operasi Bulk Berhasil.")

# Menampilkan ringkasan hasil operasi
print(f"Inserted: {result.inserted_count}")
print(f"Modified: {result.modified_count}")
print(f"Deleted: {result.deleted_count}")

# === TAMPILKAN HASIL DATA TERKINI ===
print("\n=== Current Courses Data ===")
for course in courses_collection.find():
    print(course)

Eksekusi Operasi Bulk Berhasil.
Inserted: 4
Modified: 2
Deleted: 1

=== Current Courses Data ===
{'_id': ObjectId('690bfc28adf64547a184eca9'), 'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics'}
{'_id': ObjectId('690bfc28adf64547a184ecaa'), 'course': 'CS 102', 'enrollments': 30, 'department': 'Computer Science'}
{'_id': ObjectId('690bfc28adf64547a184ecac'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}


In [10]:
# 2. Complex Filtering and Querying
from pymongo import MongoClient

# Koneksi ke MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

print("=== Exercise 2: Complex Filtering and Querying ===\n")

# === LATIHAN 2: Filter courses with enrollments > 20 ===
print("Mata kuliah yang memiliki lebih dari 20 mahasiswa :")
for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(f"- {course['course']} | Enrollments: {course['enrollments']} | Dept: {course['department']}")

# === TUGAS 2: Retrieve only courses from Computer Science or Mathematics ===
print("\nMata kuliah yang hanya berasal dari departemen Computer Science atau Mathematics:")
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}):
    print(f"- {course['course']} | Enrollments: {course['enrollments']} | Dept: {course['department']}")


=== Exercise 2: Complex Filtering and Querying ===

Mata kuliah yang memiliki lebih dari 20 mahasiswa :
- Math 101 | Enrollments: 35 | Dept: Mathematics
- CS 102 | Enrollments: 30 | Dept: Computer Science

Mata kuliah yang hanya berasal dari departemen Computer Science atau Mathematics:
- Math 101 | Enrollments: 35 | Dept: Mathematics
- CS 102 | Enrollments: 30 | Dept: Computer Science


In [9]:
# 3. Aggregation Framework for Data Analysis
from pymongo import MongoClient

# Koneksi ke MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

print("=== Exercise 3: Aggregation Framework for Data Analysis ===\n")

# === LATIHAN 3: Average enrollment per department ===
print("Rata-rata pendaftaran per departemen:")
avg_pipeline = [
    {
        '$group': {
            '_id': '$department',
            'average_enrollment': {'$avg': '$enrollments'}
        }
    }
]

for result in courses_collection.aggregate(avg_pipeline):
    print(f"- Department: {result['_id']} | Average Enrollment: {result['average_enrollment']:.2f}")

# === TUGAS 3: Maximum enrollment per department ===
print("\nJumlah pendaftaran maksimum per departemen:")
max_pipeline = [
    {
        '$group': {
            '_id': '$department',
            'max_enrollment': {'$max': '$enrollments'}
        }
    }
]

for result in courses_collection.aggregate(max_pipeline):
    print(f"- Department: {result['_id']} | Max Enrollment: {result['max_enrollment']}")


=== Exercise 3: Aggregation Framework for Data Analysis ===

Rata-rata pendaftaran per departemen:
- Department: Computer Science | Average Enrollment: 30.00
- Department: Physics | Average Enrollment: 15.00
- Department: Mathematics | Average Enrollment: 35.00

Jumlah pendaftaran maksimum per departemen:
- Department: Computer Science | Max Enrollment: 30
- Department: Physics | Max Enrollment: 15
- Department: Mathematics | Max Enrollment: 35


In [8]:
from pymongo import MongoClient

# Koneksi ke MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

print("=== Exercise 4: Data Transformation using $project and $addFields ===\n")

# === LATIHAN 4: Gunakan $project untuk menampilkan dan mengganti nama kolom ===
print("Projected fields (renamed columns):")
project_pipeline = [
    {
        '$project': {
            '_id': 0,  # sembunyikan _id
            'course_name': '$course',
            'department_name': '$department',
            'enrollments': 1
        }
    }
]

for doc in courses_collection.aggregate(project_pipeline):
    print(doc)

# === TUGAS 4: Gunakan $addFields untuk membuat kolom baru berdasarkan kondisi ===
print("\nAdd new field 'enrollment_category':")
addfields_pipeline = [
    {
        '$addFields': {
            'enrollment_category': {
                '$cond': {
                    'if': {'$gt': ['$enrollments', 20]},
                    'then': 'high',
                    'else': 'low'
                }
            }
        }
    },
    {
        '$project': {
            '_id': 0,
            'course': 1,
            'department': 1,
            'enrollments': 1,
            'enrollment_category': 1
        }
    }
]

for doc in courses_collection.aggregate(addfields_pipeline):
    print(doc)


=== Exercise 4: Data Transformation using $project and $addFields ===

Projected fields (renamed columns):
{'enrollments': 35, 'course_name': 'Math 101', 'department_name': 'Mathematics'}
{'enrollments': 30, 'course_name': 'CS 102', 'department_name': 'Computer Science'}
{'enrollments': 15, 'course_name': 'Physics 202', 'department_name': 'Physics'}

Add new field 'enrollment_category':
{'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics', 'enrollment_category': 'high'}
{'course': 'CS 102', 'enrollments': 30, 'department': 'Computer Science', 'enrollment_category': 'high'}
{'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'enrollment_category': 'low'}


In [11]:
# Homework 1 — Hitung jumlah course per department
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

print("=== Homework 1: Count of Courses per Department ===")

pipeline_1 = [
    {
        '$group': {
            '_id': '$department',
            'course_count': {'$sum': 1}
        }
    }
]

for result in courses_collection.aggregate(pipeline_1):
    print(f"- Department: {result['_id']} | Total Courses: {result['course_count']}")


=== Homework 1: Count of Courses per Department ===
- Department: Mathematics | Total Courses: 1
- Department: Computer Science | Total Courses: 1
- Department: Physics | Total Courses: 1


In [13]:
# Homework 2 — Filter + Group
print("\n=== Homework 2: Courses with enrollments > 25 in 'Computer Science' ===")

pipeline_2 = [
    {
        '$match': {
            'department': 'Computer Science',
            'enrollments': {'$gt': 25}
        }
    },
    {
        '$group': {
            '_id': '$department',
            'total_courses': {'$sum': 1},
            'average_enrollment': {'$avg': '$enrollments'}
        }
    }
]

for result in courses_collection.aggregate(pipeline_2):
    print(f"- Department: {result['_id']} | Total Courses: {result['total_courses']} | Avg Enrollment: {result['average_enrollment']:.2f}")



=== Homework 2: Courses with enrollments > 25 in 'Computer Science' ===
- Department: Computer Science | Total Courses: 1 | Avg Enrollment: 30.00


In [4]:
from pymongo import MongoClient

# Koneksi ke MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']
students_collection = db['students']

# === Buat contoh data students (kalau belum ada) ===
students_collection.delete_many({})
students_collection.insert_many([
    {'name': 'Alice', 'course': 'Math 101'},
    {'name': 'Bob', 'course': 'CS 102'},
    {'name': 'Charlie', 'course': 'CS 102'},
    {'name': 'Diana', 'course': 'Physics 202'}
])

# === Homework 3: $lookup join ===
print("=== Homework 3: Join Courses with Students using $lookup ===")

pipeline_3 = [
    {
        '$lookup': {
            'from': 'students',
            'localField': 'course',
            'foreignField': 'course',
            'as': 'enrolled_students'
        }
    },
    {
        '$project': {
            '_id': 0,
            'course': 1,
            'department': 1,
            'enrollments': 1,
            'enrolled_students.name': 1
        }
    }
]

for result in courses_collection.aggregate(pipeline_3):
    print(result)


=== Homework 3: Join Courses with Students using $lookup ===
{'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics', 'enrolled_students': [{'name': 'Alice'}]}
{'course': 'CS 102', 'enrollments': 30, 'department': 'Computer Science', 'enrolled_students': [{'name': 'Bob'}, {'name': 'Charlie'}]}
{'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'enrolled_students': [{'name': 'Diana'}]}
